In [58]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
import pandas as pd

In [59]:
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

# NASA Mars News

In [60]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html=browser.html
soup=bs(html,'html.parser')

In [61]:
# Retrieve the latest news title
news_title=soup.find_all('div', class_='content_title')[1].text
# Retrieve the latest news paragraph
news_p=soup.find_all('div', class_='article_teaser_body')[0].text
print(news_title)
news_p

NASA's Self-Driving Perseverance Mars Rover 'Takes the Wheel'


'The agency’s newest rover is trekking across the Martian landscape using a newly enhanced auto-navigation system.'

# JPL Mars Space Images - Featured Image

In [80]:
# Mars Image to be scraped
jpl_nasa_url = 'https://www.jpl.nasa.gov'
images_url = 'https://www.jpl.nasa.gov/images?query=mars'

browser.visit(images_url)

html = browser.html

images_soup = bs(html, 'html.parser')

In [99]:
# Retrieve featured image link
relative_image_path = images_soup.find_all('img', class_="BaseImage  object-contain")[0]
featured_image_url = relative_image_path
print(featured_image_url)
# <img data-src="https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA15964.2e16d0ba.fill-400x400-c50.jpg" alt="" loading="lazy" class="BaseImage  object-contain" src="https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA15964.2e16d0ba.fill-400x400-c50.jpg">

IndexError: list index out of range

# Mars Weather

In [65]:
# Scrape Mars weather from twitter
url='https://twitter.com/marswxreport?lang=en'
browser.visit(url)
html=browser.html
soup=bs(html,'html.parser')

In [66]:
# Retrieve the latest tweet from Mars weather twitter
mars_tweets = [soup.find_all('p', class_="TweetTextSize"), soup.find_all('span', class_="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0")]

In [67]:
for tweets in mars_tweets:
    mars_tweet = tweets

In [68]:
for tweet in mars_tweet:
    if 'InSight' in tweet.text:
        mars_weather = tweet.text
        if tweet.a in tweet:
            mars_weather = mars_weather.strip(tweet.a.text)
        break
print(mars_weather)

Don’t miss what’s happening


# Mars Facts

In [69]:
# Scrape Mars facts from https://space-facts.com/mars/
url='https://space-facts.com/mars/'
tables=pd.read_html(url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [70]:
mars_fact=tables[0]
mars_fact=mars_fact.rename(columns={0:"Profile",1:"Value"},errors="raise")
mars_fact.set_index("Profile",inplace=True)
mars_fact

,Value
Profile,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [71]:
fact_table=mars_fact.to_html()
fact_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Profile</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n  <

In [72]:
fact_table.replace('\n','')
print(fact_table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Value</th>
    </tr>
    <tr>
      <th>Profile</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Equatorial Diameter:</th>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>Polar Diameter:</th>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>Orbit Distance:</th>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>Orbit Period:</th>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>Surface Temperature:</th>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>First Record:</th>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>Recorded By:</th>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


# Mars Hemispheres

In [73]:
# Scrape Mars hemisphere title and image
usgs_url='https://astrogeology.usgs.gov'
url='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)
html=browser.html
soup=bs(html,'html.parser')

In [74]:
# Extract hemispheres item elements
mars_hems=soup.find('div',class_='collapsible results')
mars_item=mars_hems.find_all('div',class_='item')
hemisphere_image_urls=[]

In [76]:
# Loop through each hemisphere item
for item in mars_item:
    # Error handling
    try:
        # Extract title
        hem=item.find('div',class_='description')
        title=hem.h3.text
        # Extract image url
        hem_url=hem.a['href']
        browser.visit(usgs_url+hem_url)
        html=browser.html
        soup=bs(html,'html.parser')
        image_src=soup.find('li').a['href']
        if (title and image_src):
            # Print results
            print('-'*50)
            print(title)
            print(image_src)
        # Create dictionary for title and url
        hem_dict={
            'title':title,
            'image_url':image_src
        }
        hemisphere_image_urls.append(hem_dict)
    except Exception as e:
        print(e)

--------------------------------------------------
Cerberus Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
--------------------------------------------------
Schiaparelli Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
--------------------------------------------------
Syrtis Major Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
--------------------------------------------------
Valles Marineris Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg


In [77]:
# Create dictionary for all info scraped from sources above
mars_dict={
    "news_title":news_title,
    "news_p":news_p,
    "featured_image_url":featured_image_url,
    "mars_weather":mars_weather,
    "fact_table":fact_table,
    "hemisphere_images":hemisphere_image_urls
}

In [78]:
mars_dict

{'news_title': "NASA's Self-Driving Perseverance Mars Rover 'Takes the Wheel'",
 'news_p': 'The agency’s newest rover is trekking across the Martian landscape using a newly enhanced auto-navigation system.',
 'featured_image_url': '/_nuxt/img/logo-tribrand-color.8f95c3b.svg',
 'mars_weather': 'Don’t miss what’s happening',
 'fact_table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Profile</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n